# Index crypto
Base layer - BTC, ETH, SOL, BNB, XRP

Flow - ADA, DOT, AVAX, AVAX, ATOM, 

High Yield - XLM, TIA, TAO, AUG/REPV2, PEPE, DOGE, 



In [13]:
# imports
import pandas as pd
import numpy as np
import sys
import os

import matplotlib.pyplot as plt
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"

from datetime import datetime, timedelta

# Add the correct path to find __atlas__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  # Go up one level to find __atlas__

from __atlas__.kraken import KrakenRClient, KrakenWebSocketClient


In [14]:
tickers = ["XXBTZUSD", "XETHZUSD", "SOLUSD", "BNBUSD", "XXRPZUSD", "ADAUSD", "DOTUSD", "AVAXUSD", "ATOMUSD", "XXLMZUSD", "TIAUSD", "TAOUSD", "REPV2USD", "PEPEUSD", "XDGUSD"]
corr_window = 30 #  1 month before re assessing for correlation changes -  daily data

In [15]:
client = KrakenRClient()

✓ Kraken REST API Client initialized


In [16]:
price_hist = {}

for ticker in tickers:
    df = client.get_ohlc(ticker, interval='1D')
    if not df.empty:
        price_hist[f"{ticker}"] = df


In [21]:
# find cross correlations
corr_matrix = pd.DataFrame(index=tickers, columns=tickers)
for t1 in tickers:
    for t2 in tickers:
        if t1 != t2:
            corr = price_hist[t1]['close'].pct_change().corr(price_hist[t2]['close'].pct_change())
            corr_matrix.loc[t1, t2] = corr
        else:
            corr_matrix.loc[t1, t2] = 1.0

corr_matrix = corr_matrix.astype(float).round(3)

np.fill_diagonal(corr_matrix.values, np.nan)

fig = px.imshow(corr_matrix, text_auto=True, aspect="auto", title="Cryptocurrency Cross-Correlation Matrix (Full Span [2 Years])")
fig.update_layout(width=1000, height=1000)
fig.update_xaxes(tickvals=list(range(len(tickers))), ticktext=[ticker[:-3] for ticker in tickers], tickangle=0)
fig.update_yaxes(tickvals=list(range(len(tickers))), ticktext=[ticker[:-3] for ticker in tickers], tickangle=0)
fig.show()

In [18]:
# Calculate rolling correlations over the correlation window
# Using a sliding window (add 1, remove 1) to capture correlation changes for hedge ratios
rolling_corr_over_time = []

# Get the maximum length of data available
max_len = max([len(price_hist[ticker]) for ticker in tickers])

# For each day starting from corr_window onwards
for i in range(corr_window, max_len):
    # Create a rolling correlation matrix for this window
    window_corr = pd.DataFrame(index=tickers, columns=tickers)
    
    for t1 in tickers:
        for t2 in tickers:
            # Get the window slice: [i - corr_window : i]
            if len(price_hist[t1]) > i and len(price_hist[t2]) > i:
                df1_window = price_hist[t1].iloc[i-corr_window:i]
                df2_window = price_hist[t2].iloc[i-corr_window:i]
                
                if t1 != t2:
                    corr = df1_window['close'].pct_change().corr(df2_window['close'].pct_change())
                    window_corr.loc[t1, t2] = corr
                else:
                    window_corr.loc[t1, t2] = np.nan
    
    window_corr = window_corr.astype(float).round(3)
    rolling_corr_over_time.append({
        'date': price_hist[tickers[0]].index[i],
        'corr_matrix': window_corr.copy()
    })

print(f"Calculated {len(rolling_corr_over_time)} rolling correlation windows")
print(f"First window date: {rolling_corr_over_time[0]['date']}")
print(f"Last window date: {rolling_corr_over_time[-1]['date']}")
print("\nMost recent correlation matrix:")
print(rolling_corr_over_time[-1]['corr_matrix'])

Calculated 691 rolling correlation windows
First window date: 2023-12-09 00:00:00
Last window date: 2025-10-29 00:00:00

Most recent correlation matrix:
          XXBTZUSD  XETHZUSD  SOLUSD  BNBUSD  XXRPZUSD  ADAUSD  DOTUSD  \
XXBTZUSD       NaN     0.929   0.914     NaN     0.885   0.933   0.917   
XETHZUSD     0.929       NaN   0.926     NaN     0.874   0.962   0.925   
SOLUSD       0.914     0.926     NaN     NaN     0.866   0.940   0.911   
BNBUSD         NaN       NaN     NaN     NaN       NaN     NaN     NaN   
XXRPZUSD     0.885     0.874   0.866     NaN       NaN   0.922   0.930   
ADAUSD       0.933     0.962   0.940     NaN     0.922     NaN   0.978   
DOTUSD       0.917     0.925   0.911     NaN     0.930   0.978     NaN   
AVAXUSD      0.826     0.844   0.795     NaN     0.905   0.907   0.916   
ATOMUSD      0.848     0.900   0.877     NaN     0.909   0.955   0.948   
XXLMZUSD     0.914     0.901   0.884     NaN     0.937   0.931   0.932   
TIAUSD       0.841     0.887   0.

In [ ]:
# find cross correlations
corr_matrix = pd.DataFrame(index=tickers, columns=tickers)
for t1 in tickers:
    for t2 in tickers:
        if t1 != t2:
            corr = price_hist[t1]['close'].pct_change().corr(price_hist[t2]['close'].pct_change())
            corr_matrix.loc[t1, t2] = corr
        else:
            corr_matrix.loc[t1, t2] = 1.0

corr_matrix = corr_matrix.astype(float).round(3)

np.fill_diagonal(corr_matrix.values, np.nan)

fig = px.imshow(corr_matrix, text_auto=True, aspect="auto", title="Cryptocurrency Cross-Correlation Matrix (Latest Window [30 days])")
fig.update_layout(width=1000, height=1000)
fig.update_xaxes(tickvals=list(range(len(tickers))), ticktext=[ticker[:-3] for ticker in tickers], tickangle=0)
fig.update_yaxes(tickvals=list(range(len(tickers))), ticktext=[ticker[:-3] for ticker in tickers], tickangle=0)
fig.show()

In [19]:
# use rolling correlations to compute hedge ratios against BTC
# hedge ratio = correlation * (std_dev_asset / std_dev_btc)
# This tells us how much of each asset we need to short BTC to hedge

hedge_ratios_over_time = []
base_ticker = 'XXBTZUSD'

for window_data in rolling_corr_over_time:
    date = window_data['date']
    corr_matrix = window_data['corr_matrix']
    hedge_ratios = {'date': date}
    
    # Calculate rolling volatilities for this window
    btc_window_idx = price_hist[base_ticker].index.get_loc(date)
    btc_pct_change = price_hist[base_ticker].iloc[btc_window_idx-corr_window:btc_window_idx]['close'].pct_change()
    btc_vol = btc_pct_change.std()
    
    for ticker in tickers:
        if ticker != base_ticker:
            # Get correlation to BTC
            corr_to_btc = corr_matrix.loc[ticker, base_ticker]
            
            # Get volatility of this asset
            try:
                ticker_window_idx = price_hist[ticker].index.get_loc(date)
                ticker_pct_change = price_hist[ticker].iloc[ticker_window_idx-corr_window:ticker_window_idx]['close'].pct_change()
                ticker_vol = ticker_pct_change.std()
                
                # Calculate hedge ratio (how much to short BTC per 1 unit long of this asset)
                if not pd.isna(corr_to_btc) and btc_vol > 0:
                    hedge_ratio = corr_to_btc * (ticker_vol / btc_vol)
                else:
                    hedge_ratio = np.nan
            except:
                hedge_ratio = np.nan
            
            hedge_ratios[ticker] = hedge_ratio
    
    hedge_ratios_over_time.append(hedge_ratios)

# Convert to DataFrame for easier manipulation
hedge_ratios_df = pd.DataFrame(hedge_ratios_over_time)
hedge_ratios_df.set_index('date', inplace=True)

print("Rolling Hedge Ratios against BTC:")
print(f"Shape: {hedge_ratios_df.shape}")
print(f"\nMost recent hedge ratios (short BTC per 1 unit long):")
print(hedge_ratios_df.iloc[-1].round(3))
print(f"\nHedge ratio statistics:")
print(hedge_ratios_df.describe().round(3))

Rolling Hedge Ratios against BTC:
Shape: (691, 14)

Most recent hedge ratios (short BTC per 1 unit long):
XETHZUSD    1.660
SOLUSD      1.976
BNBUSD        NaN
XXRPZUSD    1.592
ADAUSD      2.155
DOTUSD      2.299
AVAXUSD     2.204
ATOMUSD     2.048
XXLMZUSD    1.726
TIAUSD      2.948
TAOUSD        NaN
REPV2USD    1.430
PEPEUSD     2.540
XDGUSD      2.446
Name: 2025-10-29 00:00:00, dtype: float64

Hedge ratio statistics:
       XETHZUSD   SOLUSD  BNBUSD  XXRPZUSD   ADAUSD   DOTUSD  AVAXUSD  \
count   691.000  691.000     0.0   691.000  691.000  691.000  691.000   
mean      1.194    1.396     NaN     1.052    1.410    1.254    1.490   
std       0.373    0.345     NaN     0.501    0.580    0.421    0.390   
min       0.482    0.524     NaN     0.042    0.383    0.116    0.495   
25%       0.939    1.147     NaN     0.662    1.026    0.977    1.228   
50%       1.119    1.357     NaN     0.925    1.337    1.166    1.496   
75%       1.410    1.570     NaN     1.451    1.630    1.579    

In [31]:
# assemble portfolio based on hedge ratios (sizing) and rolling correlations (beta - l/s)
# Strategy: Equal-weight long positions, hedged with BTC short positions based on hedge ratios

# Define equal weights for long positions (exclude BTC, will be short)
long_tickers = [t for t in tickers if t != 'XXBTZUSD']
equal_weight_long = 1.0 / len(long_tickers)

# Calculate portfolio weights over time
portfolio_weights_over_time = []

for idx, row in hedge_ratios_df.iterrows():
    weights = {'date': idx}
    total_btc_hedge = 0
    
    # Long positions with equal weight
    for ticker in long_tickers:
        weights[f'LONG_{ticker}'] = equal_weight_long
        # Add to total BTC hedge requirement
        hedge_ratio = row[ticker]
        if not pd.isna(hedge_ratio):
            total_btc_hedge += equal_weight_long * hedge_ratio
    
    # Short BTC position (negative weight = short)
    weights['SHORT_XXBTZUSD'] = -total_btc_hedge
    weights['NET_CASH'] = 1.0 - (len(long_tickers) * equal_weight_long + total_btc_hedge)
    
    portfolio_weights_over_time.append(weights)

# Convert to DataFrame
portfolio_weights_df = pd.DataFrame(portfolio_weights_over_time)
portfolio_weights_df.set_index('date', inplace=True)

# Actual Value
portfolio_values_df = pd.DataFrame(index=portfolio_weights_df.index)
for col in portfolio_weights_df.columns:
    if col != "NET_CASH":
        portfolio_values_df[col] = portfolio_weights_df[col] * price_hist[col.split("_")[1]]['close']
    else:
        portfolio_values_df = portfolio_weights_df
    

print("Portfolio Allocation Over Time:")
print(f"Shape: {portfolio_weights_df.shape}")
print(f"\nMost recent portfolio weights:")
print(portfolio_weights_df.iloc[-1].round(4))
print(f"\nLong exposure: {portfolio_weights_df.iloc[-1][[c for c in portfolio_weights_df.columns if c.startswith('LONG')]].sum():.4f}")
print(f"Short BTC exposure: {portfolio_weights_df.iloc[-1]['SHORT_XXBTZUSD']:.4f}")
print(f"Net cash: {portfolio_weights_df.iloc[-1]['NET_CASH']:.4f}")

# Visualize evolution of portfolio hedging
# Create subplots: 1 col, 4 rows
fig = make_subplots(rows=4, cols=1, 
                    subplot_titles=('Price & Indicators', 'RSI', 'ADX', 'Price & Signals'),
                    vertical_spacing=0.1,
                    row_heights=[0.4, 0.2, 0.2, 0.2])

# Row 1: Long exposure
fig.add_trace(
    go.Scatter(x=portfolio_weights_df.index, 
               y=portfolio_weights_df[[c for c in portfolio_weights_df.columns if c.startswith('LONG')]].sum(axis=1),
               name='Total Long Exposure', 
               line=dict(color='green', width=2)),
    row=1, col=1
)

# Add BTC Short exposure (absolute value for visualization)
fig.add_trace(
    go.Scatter(x=portfolio_weights_df.index, 
               y=-portfolio_weights_df['SHORT_XXBTZUSD'],
               name='BTC Short Hedge', 
               line=dict(color='red', width=2)),
    row=1, col=1
)

# Add Net cash
fig.add_trace(
    go.Scatter(x=portfolio_weights_df.index, 
               y=portfolio_weights_df['NET_CASH'],
               name='Net Cash', 
               line=dict(color='gray', width=1, dash='dash')),
    row=1, col=1
)

# Row 2: RSI (placeholder)
# TODO: Add RSI calculation and plotting

# Row 3: ADX (placeholder)
# TODO: Add ADX calculation and plotting

# Row 4: Price & Signals (placeholder)
# TODO: Add price with buy/sell signals

fig.update_layout(
    title="Portfolio Evolution: Long Assets vs BTC Hedge",
    showlegend=True,
    height=1200,  # Increased height for 4 subplots
    width=1200,
    hovermode='x unified'
)

# Update y-axes titles
fig.update_yaxes(title_text="Weight", row=1, col=1)
fig.update_yaxes(title_text="RSI", row=2, col=1)
fig.update_yaxes(title_text="ADX", row=3, col=1)
fig.update_yaxes(title_text="Price", row=4, col=1)

fig.show()

Portfolio Allocation Over Time:
Shape: (691, 16)

Most recent portfolio weights:
LONG_XETHZUSD     0.0714
LONG_SOLUSD       0.0714
LONG_BNBUSD       0.0714
LONG_XXRPZUSD     0.0714
LONG_ADAUSD       0.0714
LONG_DOTUSD       0.0714
LONG_AVAXUSD      0.0714
LONG_ATOMUSD      0.0714
LONG_XXLMZUSD     0.0714
LONG_TIAUSD       0.0714
LONG_TAOUSD       0.0714
LONG_REPV2USD     0.0714
LONG_PEPEUSD      0.0714
LONG_XDGUSD       0.0714
SHORT_XXBTZUSD   -1.7874
NET_CASH         -1.7874
Name: 2025-10-29 00:00:00, dtype: float64

Long exposure: 1.0000
Short BTC exposure: -1.7874
Net cash: -1.7874


In [28]:
portfolio_weights_df

,LONG_XETHZUSD,LONG_SOLUSD,LONG_BNBUSD,LONG_XXRPZUSD,LONG_ADAUSD,LONG_DOTUSD,LONG_AVAXUSD,LONG_ATOMUSD,LONG_XXLMZUSD,LONG_TIAUSD,LONG_TAOUSD,LONG_REPV2USD,LONG_PEPEUSD,LONG_XDGUSD,SHORT_XXBTZUSD,NET_CASH
date,,,,,,,,,,,,,,,,
2023-12-09,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,-0.906886,-0.906886
2023-12-10,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,-0.879458,-0.879458
2023-12-11,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,-0.883890,-0.883890
2023-12-12,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,-0.890798,-0.890798
2023-12-13,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,-0.862544,-0.862544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-25,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,-1.763745,-1.763745
2025-10-26,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,-1.753482,-1.753482
2025-10-27,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,-1.722161,-1.722161


In [ ]:
# Track how correlations evolve for specific pairs
btc_eth_corr_over_time = [rc['corr_matrix'].loc['XXBTZUSD', 'XETHZUSD'] 
                          for rc in rolling_corr_over_time]
dates = [rc['date'] for rc in rolling_corr_over_time]

fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=btc_eth_corr_over_time, mode='lines', 
                         name='BTC-ETH Rolling Correlation'))
fig.update_layout(title="30-Day Rolling Correlation: BTC vs ETH", 
                  xaxis_title="Date", yaxis_title="Correlation")
fig.show()

In [ ]:
# plot all on one chart
fig = go.Figure()
for ticker in tickers:
    fig.add_trace(go.Scatter(x=price_hist[ticker].index, y=price_hist[ticker]['close'], mode='lines', name=ticker[:-3]))

fig.add
fig.update_layout(title="Cryptocurrency Prices Over Time", xaxis_title="Date", yaxis_title="Price (USD)", width=1200, height=600)
fig.show()